# Data Collection (Zbieranie Danych)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
headers = {
    'referer': 'https://www.scrapingcourse.com/ecommerce/',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/json',
    'accept-encoding': 'gzip, deflate, br',
    'sec-ch-device-memory': '8',
    'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    'sec-ch-ua-platform': "Windows",
    'sec-ch-ua-platform-version': '"10.0.0"',
    'sec-ch-viewport-width': '792',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
  }

In [3]:
def get_page(url):
    response = requests.get(url, headers= headers, timeout=10)
    if response.status_code ==200:
        return response.text
    else:
        print(f"Błąd {response.status_code} przy pobraniu strony {url}")
        return None

In [35]:
def scrape_table(link):
    if 'otodom.pl' in link:
        #html_content = get_page(link)
        #soup = BeautifulSoup(html_content, 'html.parser')
        tab = 'brak opisu otodom'
        description = 'brak opisu otodom'
        return  tab, description
            
    elif 'olx.pl' in link:
        html_content = get_page(link)
        soup = BeautifulSoup(html_content, 'html.parser')
        tab_element = soup.find('div', class_='css-41yf00')
        description_element = soup.find('div', class_="css-19duwlz")

        #Sprawdzenie czy elementy istnieją przed użyciem .text
        tab = tab_element.text.strip() if tab_element else 'brak danych'
        description = description_element.text.strip() if description_element else 'brak opisu'
        return tab, description
    else:
        tab = 'brak opisu'
        description = 'brak opisu'
        return  tab, description

In [37]:
def fix_link(href):
    # Jeśli link zaczyna się od 'https', to już jest pełny link
    if href.startswith('https'):
        return href
    # Jeśli to link relatywny, dodaj https://www.olx.pl
    else:
        return 'https://www.olx.pl' + href

In [39]:
def notice(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    result_container = soup.find_all('div', class_="css-l9drzq")
    
    data =[]
    
    for offer in result_container:
        try:
            title = offer.find('h4').text.strip() if offer.find('h4') else "None"
            price = offer.find('p', class_='css-6j1qjp').text.strip() if offer.find('p', class_='css-6j1qjp') else "None"
            area = offer.find("span",class_="css-6as4g5").text.strip()  if offer.find('span', class_='css-6as4g5') else "None"
            address = offer.find("p",class_="css-1mwdrlh").text.strip() if offer.find('p', class_='css-1mwdrlh') else "Brak adresu"
            link = offer.find('a').get('href') if offer.find('a') else "Brak linku"
            link = fix_link(link)
            tab, description = scrape_table(link)
        
            data.append({
                    'Title': title,
                    'Price': price,
                    'Area': area,
                    'Address': address,
                    'description': description,
                    'table': tab,
                    'Link': link
                
            })  
            
        except Exception as e:
                print(f"Błąd przy przetwarzaniu oferty: {e}")
    return data
    


In [41]:
BASE_URL = "https://www.olx.pl/nieruchomosci/mieszkania/wynajem/krakow/?page="

In [43]:
def scrape_multiple_pages(pages=5):
    all_listings = []
    for page in range(1, pages + 1):
        print(f"Scrapowanie strony {page}")
        url = BASE_URL + str(page)
        html_content = get_page(url)
        if html_content:
            listings = notice(html_content)
            all_listings.extend(listings)
    return all_listings

In [51]:
if __name__ == "__main__":
    data = scrape_multiple_pages(25)
    df = pd.DataFrame(data)
    df.to_csv('data_raw\olx_krakow_detailed_listings.csv', index=False)

Scrapowanie strony 1
Scrapowanie strony 2
Scrapowanie strony 3
Scrapowanie strony 4
Scrapowanie strony 5
Scrapowanie strony 6
Scrapowanie strony 7
Scrapowanie strony 8
Scrapowanie strony 9
Scrapowanie strony 10
Scrapowanie strony 11
Scrapowanie strony 12
Scrapowanie strony 13
Scrapowanie strony 14
Scrapowanie strony 15
Scrapowanie strony 16
Scrapowanie strony 17
Scrapowanie strony 18
Scrapowanie strony 19
Scrapowanie strony 20
Scrapowanie strony 21
Scrapowanie strony 22
Scrapowanie strony 23
Scrapowanie strony 24
Scrapowanie strony 25


In [53]:
df

,Title,Price,Area,Address,description,table,Link
0,Mieszkanie 3-pokojowe - ul. Bajeczna 2B - Grze...,3 250 zł,"68,40 m²","Kraków, Grzegórzki - Odświeżono dnia 24 marca ...",Do wynajęcia trzypokojowe mieszkanie o powierz...,FirmoweZwierzęta: NieWinda: TakPoziom: 4Umeblo...,https://www.olx.pl/oferta/mieszkanie-3-pokojow...
1,NOWE/Klima/ 38m2/2 pokoje/ wysoki standard/mie...,3 300 złdo negocjacji,38 m²,"Kraków, Czyżyny - 23 marca 2025",MIESZKANIE:\nDo wynajęcia nowocześnie urządzon...,PrywatneZwierzęta: NieWinda: TakParking: przyn...,https://www.olx.pl/oferta/nowe-klima-38m2-2-po...
2,Mieszkanie z klimatyzacją od zaraz | Dąbska | ...,2 800 zł,40 m²,"Kraków, Grzegórzki - 24 marca 2025",ENG BELOW\nPromo Estates przedstawia ofertę wy...,FirmoweZwierzęta: TakWinda: TakParking: w gara...,https://www.olx.pl/oferta/mieszkanie-z-klimaty...
3,Nowoczesna kawalerka ul. Monte Cassino /100m o...,2 100 zł,20 m²,"Kraków, Stare Miasto - Odświeżono Dzisiaj o 10:10",English down below.\n\nPrezentuję na wynajem n...,PrywatneZwierzęta: NieWinda: NiePoziom: 1Umebl...,https://www.olx.pl/oferta/nowoczesna-kawalerka...
4,Studio pierwszy najem 2300 zł Ruczaj,2 300 zł,22 m²,"Kraków, Dębniki - Odświeżono Dzisiaj o 10:19",Cena 2600 zł z czynszem + indywidualne opłaty ...,PrywatneZwierzęta: TakWinda: TakParking: przyn...,https://www.olx.pl/oferta/studio-pierwszy-naje...
...,...,...,...,...,...,...,...
1291,2 Pokoje - Parking - Osobna Kuchnia - Wietora,2 900 zł,42 m²,"Kraków, Stare Miasto - 26 lutego 2025",brak opisu otodom,brak opisu otodom,https://www.otodom.pl/pl/oferta/2-pokoje-parki...
1292,2 Pokoje! - Nowa 5 Dzielnica ul. Wrocławska EN...,3 585 zł,45 m²,"Kraków, Krowodrza - 21 marca 2025",brak opisu otodom,brak opisu otodom,https://www.otodom.pl/pl/oferta/2-pokoje-nowa-...
1293,Kawalerka 35 m2 Czyżyny Sołtysowska Osiedle Za...,2 150 zł,35 m²,"Kraków, Czyżyny - 23 marca 2025",Wynajmę Kawalerkę na zamkniętym osiedlu Sołtys...,PrywatneZwierzęta: NieWinda: TakParking: parki...,https://www.olx.pl/oferta/kawalerka-35-m2-czyz...
1294,"Dwupokojowe mieszkanie, dostępne od zaraz",2 700 zł,38 m²,"Kraków, Mistrzejowice - 06 marca 2025",brak opisu otodom,brak opisu otodom,https://www.otodom.pl/pl/oferta/dwupokojowe-mi...
